In [28]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

In [29]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# val_it = datagen.flow_from_directory('data/validation/', class_mode='categorical')
test_it = train_datagen.flow_from_directory('data/images/test',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# confirm the iterator works
train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
test_batchX, test_batchy = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (test_batchX.shape, test_batchX.min(), test_batchX.max()))

Found 58 images belonging to 5 classes.
Found 16 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(16, 224, 224, 3), min=-123.680, max=151.061


In [33]:
# classifying only weapons 
vgg16_model = VGG16(weights="imagenet")

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation

seq_model = Sequential()

# iterate all layers in vgg16 to sequential for custom modification
for layer in vgg16_model.layers:
    seq_model.add(layer)
    
seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [35]:
# 1000 categories, pop off output layer to work only specific categories
seq_model._layers.pop()

seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [36]:
# freeze weights to prevent constantly updating weights
for layer in seq_model._layers:
    layer.trainable = False

In [40]:
seq_model.add(Dense(5, activation='softmax')) # 5 categories 

seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

C:\Users\xris1\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [38]:
seq_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_it.n//train_it.batch_size
seq_model.fit_generator(generator=train_it,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1_2/convolution}}]]
	 [[Mean_2/_877]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1_2/convolution}}]]
0 successful operations.
0 derived errors ignored.